In [1]:
import cv2
import numpy as np


faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eyeCascade=cv2.CascadeClassifier("haarcascade_eye.xml")

video_capture = cv2.VideoCapture(0)
cv2.namedWindow("Face and eyes")

while True:
    
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    
   
    

    faces = faceCascade.detectMultiScale(       #ritorna lista di rettangoli
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
        )
    
     # Disegna un rettangolo attorno al volto
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        roi_gray=gray[y:y+h/2,x:x+w]
        roi_color=frame[y:y+h/2,x:x+w]
        
        circles = cv2.HoughCircles(roi_gray,cv2.HOUGH_GRADIENT,1,20,
                            param1=50,param2=30,minRadius=0,maxRadius=0)
    
    
        circles = np.uint16(np.around(circles))
    
        for i in circles[0,:]:
                cv2.circle(roi_color,(i[0],i[1]),i[2],(0,255,0),2)
                cv2.circle(roi_color,(i[0],i[1]),2,(0,0,255),3)
            
        
        
        
        
        
        eyes= eyeCascade.detectMultiScale(         #ritorna lista di rettangoli
        roi_gray,
        scaleFactor=1.1,
        minNeighbors=4,
        minSize=(10, 10)
    )


                                                  # Disegna un rettangolo attorno agli occhi
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (255, 191, 0), 2)
            
            roi_gray1=gray[ey:ey+eh,ex:ex+ew]
            roi_color2=frame[ey:ey+eh,ex:ex+ew]
            
            
            
            
            
        
            
            
            
            

    # Mostra il video finale
    cv2.imshow('Face and eyes', frame)
    if cv2.waitKey(1)==27:
        break
    

video_capture.release()

cv2.destroyAllWindows()


C:\Users\giada\Anaconda3\lib\site-packages\ipykernel\__main__.py:32: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\giada\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
